In [ ]:
# !pip install gym
# !pip install pygame

In [ ]:
# ! pip install matplotlib

In [1]:
import gym
import numpy as np 
import matplotlib.pyplot as plt
import time

c:\Users\rossi\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\wrappers\monitoring\video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn


In [2]:
env = gym.make('CartPole-v1')
print(env.observation_space.low,"\n",env.observation_space.high)

[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38] 
 [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]


In [3]:
state_space = 4 # number of states
action_space = 2 # number of possible actions
import math
positive_infinity = math.inf
negative_infinity = -math.inf


def Qtable(state_space,action_space,bin_size = 30):
    
    bins = [np.linspace(-4.8,4.8,bin_size),
            np.linspace(negative_infinity,positive_infinity,bin_size),
            np.linspace(-0.418,0.418,bin_size),
            np.linspace(negative_infinity,positive_infinity,bin_size)]
    
    q_table = np.random.uniform(low=-1,high=1,size=([bin_size] * state_space + [action_space]))
    return q_table, bins

def Discrete(state, bins):
    index = []
    for i in range(len(state)): index.append(np.digitize(state[i],bins[i]) - 1)
    return tuple(index)

In [4]:
def Q_learning(q_table, bins, episodes = 5000, gamma = 0.95, lr = 0.1, timestep = 5000, epsilon = 0.2):
    rewards = 0
    solved = False 
    steps = 0 
    runs = [0]
    data = {'max' : [0], 'avg' : [0]}
    start = time.time()
    ep = [i for i in range(0,episodes + 1,timestep)] 

      
    
    for episode in range(1,episodes+1):
        
        current_state = Discrete(env.reset(),bins) # initial observation
        score = 0
        done = False
        temp_start = time.time()
        
        while not done:
            steps += 1 
            ep_start = time.time()
            if episode%timestep == 0:
                env.render()
                
            if np.random.uniform(0,1) < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(q_table[current_state])
            
            observation, reward, done, info = env.step(action)
            next_state = Discrete(observation,bins)

            score += reward
            

            if not done:
                max_future_q = np.max(q_table[next_state])
                current_q = q_table[current_state+(action,)]
                new_q = (1-lr)*current_q + lr*(reward + gamma*max_future_q)
                q_table[current_state+(action,)] = new_q

            current_state = next_state
            
        # End of the loop update
        else:
            rewards += score
            runs.append(score)
            #if score > 475 and steps >= 100 and solved == False: # considered as a solved:
            if score > 475 and solved == False: # considered as a solved:
                solved = True
                print('Solved in episode : {} in time {}'.format(episode, (time.time()-ep_start)))
        
        # Timestep value update
        if episode%timestep == 0:
            print('Episode : {} | Reward -> {} | Max reward : {} | Time : {}'.format(episode,rewards/timestep, max(runs), time.time() - ep_start))
            data['max'].append(max(runs))
            data['avg'].append(rewards/timestep)
            if rewards/timestep >= 475: 
                print('Solved in episode : {}'.format(episode))
            rewards, runs= 0, [0] 
            
    if len(ep) == len(data['max']):
        plt.plot(ep, data['max'], label = 'Max')
        plt.plot(ep, data['avg'], label = 'Avg')
        plt.xlabel('Episode')
        plt.ylabel('Reward')
        plt.legend(loc = "upper left")
        
    env.close()

In [5]:
# TRANING
q_table, bins = Qtable(len(env.observation_space.low), env.action_space.n)

Q_learning(q_table, bins, lr = 0.15, gamma = 0.995, episodes = 5*10**5, timestep = 1000)



c:\Users\rossi\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\function_base.py:151: RuntimeWarning: invalid value encountered in multiply
  y *= step
c:\Users\rossi\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\function_base.py:161: RuntimeWarning: invalid value encountered in add
  y += start


Episode : 1000 | Reward -> 23.428 | Max reward : 89.0 | Time : 0.021000146865844727
Episode : 2000 | Reward -> 23.278 | Max reward : 101.0 | Time : 0.01965641975402832
Episode : 3000 | Reward -> 20.364 | Max reward : 68.0 | Time : 0.020006895065307617
Episode : 4000 | Reward -> 21.418 | Max reward : 107.0 | Time : 0.019991636276245117
Episode : 5000 | Reward -> 23.736 | Max reward : 79.0 | Time : 0.020259857177734375
Episode : 6000 | Reward -> 21.43 | Max reward : 91.0 | Time : 0.018993377685546875
Episode : 7000 | Reward -> 22.207 | Max reward : 106.0 | Time : 0.022036314010620117
Episode : 8000 | Reward -> 21.571 | Max reward : 77.0 | Time : 0.02355790138244629
Episode : 9000 | Reward -> 21.023 | Max reward : 98.0 | Time : 0.02098393440246582
Episode : 10000 | Reward -> 18.93 | Max reward : 73.0 | Time : 0.020992755889892578
Episode : 11000 | Reward -> 18.272 | Max reward : 75.0 | Time : 0.022002458572387695
Episode : 12000 | Reward -> 20.157 | Max reward : 68.0 | Time : 0.0200593471

KeyboardInterrupt: 

: 